In [1]:
import pandas as pd
import numpy as np
import datetime
from sim import generate_simulation
from model_cancellations import get_otb_res, predict_cancellations
import re

pd.options.display.max_rows = 150
pd.options.display.max_columns = 400

In [2]:
DATE_FMT = "%Y-%m-%d"
h1_capacity = 187
h2_capacity = 226
AOD = "2017-08-01"
AOD_dt = pd.to_datetime(AOD)

h1_res = pd.read_pickle("pickle/h1_res.pick")
h2_res = pd.read_pickle("pickle/h2_res.pick")
h1_dbd = pd.read_pickle("pickle/h1_dbd.pick")
h2_dbd = pd.read_pickle("pickle/h2_dbd.pick")
h1_sim = pd.read_pickle("pickle/h1_sim.pick")
h2_sim = pd.read_pickle("pickle/h2_sim.pick")

In [3]:
h1_sim["TM05_Date"] = h1_sim.Date - pd.DateOffset(5)
h1_sim["TM15_Date"] = h1_sim.Date - pd.DateOffset(15)
h1_sim["TM30_Date"] = h1_sim.Date - pd.DateOffset(30)

In [4]:
h1_sim.head(3)

,DOW,RoomsOTB,RevOTB,CxlForecast,Trn_RoomsOTB,Trn_RevOTB,Trn_CxlForecast,TrnP_RoomsOTB,TrnP_RevOTB,TrnP_CxlForecast,Grp_RoomsOTB,Grp_RevOTB,Grp_CxlForecast,Cnt_RoomsOTB,Cnt_RevOTB,Cnt_CxlForecast,Date,TM05_Date,TM15_Date,TM30_Date,WeekEndDate,WE,WD,STLY_Date,Occ,RevPAR,RemSupply,ADR_OTB,Trn_ADR_OTB,TrnP_ADR_OTB,Grp_ADR_OTB,Cnt_ADR_OTB,LYA_Occ,LYA_RoomsSold,LYA_ADR,LYA_RoomRev,LYA_RevPAR,LYA_NumCancels,LYA_Trn_RoomsSold,LYA_Trn_ADR,LYA_Trn_RoomRev,LYA_Grp_RoomsSold,LYA_Grp_ADR,LYA_Grp_RoomRev,LYA_TrnP_RoomsSold,LYA_TrnP_ADR,LYA_TrnP_RoomRev,LYA_Cnt_RoomsSold,LYA_Cnt_ADR,LYA_Cnt_RoomRev,Realized_Cxls,SellingPrice,TM05_RoomsOTB,TM05_ADR,TM05_TRN_OTB,TM05_TRN_ADR,TM05_TRNP_OTB,TM05_TRNP_ADR,TM05_GRP_OTB,TM05_GRP_ADR,TM05_CNT_OTB,TM05_CNT_ADR,TM15_RoomsOTB,TM15_ADR,TM15_TRN_OTB,TM15_TRN_ADR,TM15_TRNP_OTB,TM15_TRNP_ADR,TM15_GRP_OTB,TM15_GRP_ADR,TM15_CNT_OTB,TM15_CNT_ADR,TM30_RoomsOTB,TM30_ADR,TM30_TRN_OTB,TM30_TRN_ADR,TM30_TRNP_OTB,TM30_TRNP_ADR,TM30_GRP_OTB,TM30_GRP_ADR,TM30_CNT_OTB,TM30_CNT_ADR,STLY_RoomsOTB,STLY_ADR_OTB,STLY_SellingPrice,STLY_TM05_RoomsOTB,STLY_TM15_ADR,STLY_TM15_RoomsOTB,STLY_TM05_ADR,STLY_TM30_RoomsOTB,STLY_TM30_ADR,STLY_Trn_RoomsOTB,STLY_Trn_ADR_OTB,STLY_TM05_TRN_OTB,STLY_TM05_TRN_ADR,STLY_TM15_TRN_OTB,STLY_TM15_TRN_ADR,STLY_TM30_TRN_OTB,STLY_TM30_TRN_ADR,STLY_TrnP_RoomsOTB,STLY_TrnP_ADR_OTB,STLY_TM05_TRNP_OTB,STLY_TM05_TRNP_ADR,STLY_TM15_TRNP_OTB,STLY_TM15_TRNP_ADR,STLY_TM30_TRNP_OTB,STLY_TM30_TRNP_ADR,STLY_Grp_RoomsOTB,STLY_Grp_ADR_OTB,STLY_TM05_GRP_OTB,STLY_TM05_GRP_ADR,STLY_TM15_GRP_OTB,STLY_TM15_GRP_ADR,STLY_TM30_GRP_OTB,STLY_TM30_GRP_ADR,STLY_Cnt_RoomsOTB,STLY_Cnt_ADR_OTB,STLY_TM05_CNT_OTB,STLY_TM05_CNT_ADR,STLY_TM15_CNT_OTB,STLY_TM15_CNT_ADR,STLY_TM30_CNT_OTB,STLY_TM30_CNT_ADR
2017-08-01,Tue,184.0,35239.14,18.0,151.0,29935.72,16.0,13.0,2488.24,2.0,2.0,291.66,0.0,18.0,2523.52,0.0,2017-08-01,2017-07-27,2017-07-17,2017-07-02,2017-08-06,False,False,2016-08-02,0.98,188.44,21,191.52,198.25,191.40,145.83,140.20,0.98,183.0,170.00,31110.89,166.37,81.0,159.0,175.03,27829.05,0.0,0.0,0.0,14.0,152.47,2134.60,10.0,114.72,1147.24,134,204.00,177.0,192.05,144.0,199.23,13.0,191.40,2.0,145.83,18.0,140.20,180.0,191.78,147.0,198.75,13.0,191.4,2.0,145.83,18.0,140.20,170.0,188.18,139.0,195.88,11.0,177.17,2.0,145.83,18.0,140.20,183.0,170.00,178.09,182.0,167.36,180.0,168.12,180.0,166.00,159.0,175.03,158.0,172.89,156.0,172.07,156.0,170.50,14.0,152.47,14.0,152.47,14.0,152.47,14.0,152.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,114.72,10.0,114.72,10.0,114.72,10.0,114.72
2017-08-02,Wed,186.0,36141.47,19.0,152.0,30940.93,17.0,11.0,2004.24,2.0,2.0,291.66,0.0,21.0,2904.64,0.0,2017-08-02,2017-07-28,2017-07-18,2017-07-03,2017-08-06,False,False,2016-08-03,0.99,193.27,20,194.31,203.56,182.20,145.83,138.32,0.97,182.0,173.96,31660.62,169.31,86.0,159.0,178.23,28337.88,1.0,241.0,241.0,14.0,154.47,2162.60,8.0,114.89,919.14,131,206.75,183.0,193.75,149.0,203.06,11.0,182.20,2.0,145.83,21.0,138.32,183.0,192.39,149.0,201.39,11.0,182.2,2.0,145.83,21.0,138.32,174.0,187.86,143.0,196.84,9.0,162.76,2.0,145.83,20.0,139.16,182.0,173.96,181.69,173.0,172.51,176.0,172.87,179.0,170.49,159.0,178.23,151.0,177.65,154.0,177.14,157.0,174.75,14.0,154.47,14.0,154.47,14.0,154.47,14.0,154.47,1.0,241.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,114.89,8.0,114.89,8.0,114.89,8.0,114.89
2017-08-03,Thu,179.0,35705.54,17.0,140.0,29773.21,15.0,13.0,2225.28,2.0,3.0,453.06,0.0,23.0,3253.99,0.0,2017-08-03,2017-07-29,2017-07-19,2017-07-04,2017-08-06,False,False,2016-08-04,0.96,190.94,25,199.47,212.67,171.18,151.02,141.48,0.98,183.0,182.11,33326.17,178.21,96.0,153.0,185.74,28417.71,0.0,0.0,0.0,20.0,186.56,3731.12,10.0,117.73,1177.34,140,210.33,184.0,200.31,145.0,213.27,13.0,171.18,3.0,151.02,23.0,141.48,181.0,198.69,144.0,211.53,11.0,163.2,3.0,151.02,23.0,141.48,170.0,194.54,135.0,206.31,11.0,163.20,2.0,145.83,22.0,142.39,183.0,182.11,187.83,177.0,179.67,177.0,180.40,173.0,177.01,153.0,185.74,149.0,184.72,149.0,183.85,145.0,180.79,20.0,186.56,18.0,179.47,18.0,179.47,18.0,179.47,0.0,0.0,0.0,0.0,0.0,

In [5]:
# h1_stly_sim = generate_simulation(h1_dbd, "2016-08-02", 1, h1_res)

In [6]:
h1_tm_otb_cols = [col for col in h1_sim.columns if re.search(r"[Tt]?[Mm]?[0-9]{1,2}_OTB", col) is not None]
# h1_tm_stly_cols = [col for col in h1_sim.columns if re.search(r"STLY.*[Tt][Mm][0-9]{1,2}_OTB", col) is not None]
# h1_tm_cols = h1_tm_otb_cols + h1_tm_stly_cols
h1_tm_otb_cols

[]

In [7]:
np.random.normal()

-2.823571081513233

## Add Price Columns

In [8]:
night = '2017-08-12'
h1_pricing = h1_res.copy()
mask = (h1_pricing.ArrivalDate <=night) & (h1_pricing.CheckoutDate > night) & (h1_pricing.CustomerType == 'Transient')
price = h1_pricing[mask].ADR.mean()

## Add Pace Columns

In [9]:
stly_cols = [col for col in list(h1_sim.columns) if col[:4] == 'STLY' and 'TM' not in col]
stly_cols

['STLY_Date',
 'STLY_RoomsOTB',
 'STLY_ADR_OTB',
 'STLY_SellingPrice',
 'STLY_Trn_RoomsOTB',
 'STLY_Trn_ADR_OTB',
 'STLY_TrnP_RoomsOTB',
 'STLY_TrnP_ADR_OTB',
 'STLY_Grp_RoomsOTB',
 'STLY_Grp_ADR_OTB',
 'STLY_Cnt_RoomsOTB',
 'STLY_Cnt_ADR_OTB']

In [10]:
stly_segs = [('STLY_RoomsOTB', 'STLY_ADR_OTB', 'All_'),
             ('STLY_Trn_RoomsOTB','STLY_Trn_ADR_OTB', 'Trn_'),
             ('STLY_TrnP_RoomsOTB', 'STLY_TrnP_ADR_OTB', 'TrnP_'),
             ('STLY_Grp_RoomsOTB', 'STLY_Grp_ADR_OTB', 'Grp_'),
             ('STLY_Cnt_RoomsOTB','STLY_Cnt_ADR_OTB', 'Cnt_'),
]

for Rooms, ADR, name in stly_segs:
    h1_sim["STLY_" + name + "Rev"] = h1_sim[Rooms] * h1_sim[ADR]

stly_cols = [col for col in list(h1_sim.columns) if col[:4] == 'STLY' and 'TM' not in col]
h1_sim[stly_cols].head()


,STLY_Date,STLY_RoomsOTB,STLY_ADR_OTB,STLY_SellingPrice,STLY_Trn_RoomsOTB,STLY_Trn_ADR_OTB,STLY_TrnP_RoomsOTB,STLY_TrnP_ADR_OTB,STLY_Grp_RoomsOTB,STLY_Grp_ADR_OTB,STLY_Cnt_RoomsOTB,STLY_Cnt_ADR_OTB,STLY_All_Rev,STLY_Trn_Rev,STLY_TrnP_Rev,STLY_Grp_Rev,STLY_Cnt_Rev
2017-08-01,2016-08-02,183.0,170.00,178.09,159.0,175.03,14.0,152.47,0.0,0.0,10.0,114.72,31110.00,27829.77,2134.58,0.0,1147.20
2017-08-02,2016-08-03,182.0,173.96,181.69,159.0,178.23,14.0,154.47,1.0,241.0,8.0,114.89,31660.72,28338.57,2162.58,241.0,919.12
2017-08-03,2016-08-04,183.0,182.11,187.83,153.0,185.74,20.0,186.56,0.0,0.0,10.0,117.73,33326.13,28418.22,3731.20,0.0,1177.30
2017-08-04,2016-08-05,184.0,188.08,193.42,150.0,190.21,25.0,202.01,0.0,0.0,9.0,113.97,34606.72,28531.50,5050.25,0.0,1025.73
2017-08-05,2016-08-06,180.0,190.06,195.12,148.0,192.95,23.0,201.26,0.0,0.0,9.0,113.97,34210.80,28556.60,4628.98,0.0,1025.73


In [11]:
h1_sim["RoomsPace"] = h1_sim.RoomsOTB - h1_sim.STLY_RoomsOTB
h1_sim["RoomsPace"] = h1_sim.RoomsOTB - h1_sim.STLY_RoomsOTB

## Add Gap to LYA Columns

In [12]:
h1_sim["RoomsGapToLYA"] = h1_sim.LYA_RoomsSold - h1_sim.RoomsOTB
h1_sim["ADR_GapToLYA"] = h1_sim.LYA_ADR - h1_sim.ADR_OTB

## Add Pickup columns (vs T-30, T-15, T-5)

In [ ]:
# examples: "TM05_TRN_ADR","Trn_ADR_OTB","RoomsOTB", "ADR_OTB",

# this cell is ugly, but not as ugly as the one below :)

tms = ["TM30", "TM15", "TM05"]
segs = ["TRN", "TRNP", "GRP", "CNT"]

for tm in tms:
    # add total hotel stats first
    h1_sim[tm + '_RoomsPickup'] = h1_sim["RoomsOTB"] - h1_sim[tm + "_RoomsOTB"]
    h1_sim[tm + '_RoomsPickup'] = h1_sim["ADR_OTB"] - h1_sim[tm + "_ADR"]
    for seg in segs:
        # and now segmented stats    
        h1_sim[tm + '_' + seg + '_RoomsPickup'] = h1_sim[tm + '_' + seg + '_' + "OTB"] - h1_sim[tm + '_' + seg + "_OTB"]
        h1_sim[tm + '_' + seg + '_RoomsPickup'] = h1_sim[tm + '_' + seg + '_' + "ADR"] - h1_sim[tm + '_' + seg + "_ADR"]
h1_sim.head()

Above cell may be a bit overkill, but I was trying to avoid housing the below in a script in case the columns change.

In [14]:
h1_sim["TM30_RoomsPickup"] = h1_sim.RoomsOTB - h1_sim.TM30_RoomsOTB
h1_sim["TM30_ADR_Pickup"] = h1_sim.ADR_OTB - h1_sim.TM30_ADR
h1_sim["TM15_RoomsPickup"] = h1_sim.RoomsOTB - h1_sim.TM15_RoomsOTB
h1_sim["TM15_ADR_Pickup"] = h1_sim.ADR_OTB - h1_sim.TM15_ADR
h1_sim["TM05_RoomsPickup"] = h1_sim.RoomsOTB - h1_sim.TM05_RoomsOTB
h1_sim["TM05_ADR_Pickup"] = h1_sim.ADR_OTB - h1_sim.TM05_ADR
h1_sim["STLY_TM30_RoomsPickup"] = h1_sim.STLY_RoomsOTB - h1_sim.STLY_TM30_RoomsOTB
h1_sim["STLY_TM30_ADR_Pickup"] = h1_sim.STLY_ADR_OTB - h1_sim.STLY_TM30_ADR
h1_sim["STLY_TM15_RoomsPickup"] = h1_sim.STLY_RoomsOTB - h1_sim.STLY_TM15_RoomsOTB
h1_sim["STLY_TM15_ADR_Pickup"] = h1_sim.STLY_ADR_OTB - h1_sim.STLY_TM15_ADR
h1_sim["STLY_TM05_RoomsPickup"] = h1_sim.STLY_RoomsOTB - h1_sim.STLY_TM05_RoomsOTB
h1_sim["STLY_TM05_ADR_Pickup"] = h1_sim.STLY_ADR_OTB - h1_sim.STLY_TM05_ADR

# add t- rooms & ADR pickup vs STLY 
h1_sim["TM30_RoomsPickupVsSTLY"] = h1_sim.TM30_RoomsPickup - h1_sim.STLY_TM30_RoomsPickup
h1_sim["TM30_ADR_PickupVsSTLY"] = h1_sim.TM30_ADR_Pickup - h1_sim.STLY_TM30_ADR_Pickup
h1_sim["TM15_RoomsPickupVsSTLY"] = h1_sim.TM15_RoomsPickup - h1_sim.STLY_TM15_RoomsPickup
h1_sim["TM15_ADR_PickupVsSTLY"] = h1_sim.TM15_ADR_Pickup - h1_sim.STLY_TM15_ADR_Pickup
h1_sim["TM05_RoomsPickupVsSTLY"] = h1_sim.TM05_RoomsPickup - h1_sim.STLY_TM05_RoomsPickup
h1_sim["TM05_ADR_PickupVsSTLY"] = h1_sim.TM05_ADR_Pickup - h1_sim.STLY_TM05_ADR_Pickup